In [1]:
import numpy as np 
import torch
import torch.nn as nn
import torch.nn.functional as F
from google.colab import files
import pickle
from torch.utils.data import TensorDataset, DataLoader

import csv
import json
import pandas as pd
from collections import Counter
import spacy
import re
nlp = spacy.load('en_core_web_sm')
#stopwords = nlp.Defaults.stop_words

## Tweets pre-processing

In [2]:
files.upload()

Saving X_train_input.csv to X_train_input.csv
Saving X_test_meta.csv to X_test_meta.csv
Saving X_train_meta.csv to X_train_meta.csv
Saving X_test_input.csv to X_test_input.csv


{'X_test_input.csv': b'0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99\n0,1342,26,1305,194,662,1385,651,907,3,276,3,649,1317,1135,1380,3,1385,201,1126,1293,935,811,886,178,531,750,828,653,961,1331,886,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1\n0,1342,332,950,3,1049,750,3,930,166,750,3,3,1196,478,987,3,1196,3,987,3,886,188,1041,269,194,662,1385,777,258,58,1254,1143,886,1003,930,960,3,25,490,461,3,1049,212,1354,1196,1307,777,3,886,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1\n0,906,750,1342,194,662,1385,399,1126,363,464,502,1078,891,840,399,3,399,3,750,3,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,

In [3]:
files.upload()

Saving HW_TRAINING_ERRORS.csv to HW_TRAINING_ERRORS.csv
Saving MA_TRAINING_ERRORS.csv to MA_TRAINING_ERRORS.csv


{'HW_TRAINING_ERRORS.csv': b'0\n-4411.830048693955\n-3168.322558431927\n144722.2215495932\n5461.629724835902\n-5263.007767053095\n-5179.994377377647\n5300.372785006609\n-4463.065149624483\n1668.8023004287388\n-3431.0001372221805\n26154.132861863603\n445.7687798174975\n13939.877872122654\n-3247.6449610703853\n-5750.381707823465\n-1666.56326344929\n16134.262458843588\n-4440.669548591064\n-3406.8304799207035\n4242.696915454222\n-2143.8833611859127\n-1296.27528754785\n1796.7107647446505\n-2052.3959269682905\n2871.593322419141\n25904.82671403173\n-3302.992754391\n-2385.1157182586944\n-1623.2248580143819\n-1703.2814872058816\n-1221.5666296185134\n1133.4891629735139\n-905.8249029003462\n532.5733854318759\n11888.72279166943\n207.06894161573496\n750.8426545435748\n2098.888953072756\n2021.5427292620504\n3201.494602044778\n3582.3811846188705\n1082.4740367348022\n8835.155789971344\n-246.68724031685406\n114349.13869992844\n-2791.5510663377418\n-1309.830909038892\n-2319.1151160796458\n18304.18277523

In [19]:
X_test_meta = pd.read_csv('X_test_meta.csv')
X_test = pd.read_csv('X_test_input.csv')

In [20]:
X_test_meta = X_test_meta.to_numpy()[:, 1:].astype(np.int64)
X_test = X_test.to_numpy().astype(np.int64)

In [21]:
X_train_meta = pd.read_csv('X_train_meta.csv')
X_train = pd.read_csv('X_train_input.csv')

In [22]:
X_train_meta = X_train_meta.to_numpy()[:, 1:].astype(np.int64)
X_train = X_train.to_numpy().astype(np.int64)

In [23]:
exp_residuals = pd.read_csv('HW_TRAINING_ERRORS.csv')
MA_residuals = pd.read_csv('MA_TRAINING_ERRORS.csv')

In [24]:
exp_residuals = exp_residuals.values.flatten().astype(np.float32)
MA_residuals = MA_residuals.values.flatten().astype(np.float32)

In [143]:
1125/5

225.0

In [142]:
205/5

41.0

In [73]:
from torch.utils.data import DataLoader, TensorDataset


# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(X_train_meta), torch.from_numpy(MA_residuals))
test_data = TensorDataset(torch.from_numpy(X_test), torch.from_numpy(X_test_meta))

# dataloaders
batch_size = 25

train_loader = DataLoader(train_data, shuffle=False, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=False, batch_size=1)

# Neural Net

### Retweet Network: Takes in a tweet as input, can use embedded version, and can any combination of bidirectional, LSTM, GRU, concatenates it with metadata vector, and uses a feedforward neural net with 1 hidden layer to perform a regression prediction on the retweet count. 

#### Parameter custom_embeddings is either a tuple: (weight_matrix , none_trainable), or None.
#### none_trainable is either True or False or Nothing

In [74]:
def create_emb_layer(weights_matrix, non_trainable=False):
  num_embeddings, embedding_dim = weights_matrix.shape
  emb_layer = nn.Embedding(num_embeddings, embedding_dim)
  emb_layer.load_state_dict({'weight': torch.from_numpy(weights_matrix)})
  if non_trainable:
      emb_layer.weight.requires_grad = False
  return emb_layer, num_embeddings, embedding_dim

class RetweetNet(nn.Module):
  def __init__(self, vocab_size, hidden_state_sizes, meta_data_len, output_size, embedding_dim, hidden_dim, 
                 n_layers, drop_prob=0.3, custom_embeddings = None, bidirectional = False, GRU = False):
    super().__init__()
    self.GRU_val = GRU
    self.bidirectional = bidirectional
    self.output_size = output_size
    self.n_layers = n_layers
    self.hidden_dim = hidden_dim
        
    if custom_embeddings is None: 
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
    else: 
        assert len(custom_embeddings) == 2 and isinstance(custom_embeddings, tuple), "custom embeddings must be of form: (weight_matrix, non_trainable)"
        weights_matrix, non_trainable = custom_embeddings
        self.embedding, num_embeddings, embedding_dim = create_emb_layer(weights_matrix, non_trainable)
        
    if GRU == False: 
        self.Gate = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=drop_prob, batch_first=True, bidirectional = bidirectional)
    else: 
        self.Gate = nn.GRU(embedding_dim, hidden_dim, n_layers, 
                              dropout=drop_prob, batch_first=True, bidirectional = bidirectional)
    self.dropout = nn.Dropout(0.2)
    self.fc1 = nn.Linear(hidden_dim, hidden_state_sizes[0])
    self.relu = nn.ReLU()
    self.sigmoid = nn.Sigmoid()
    self.tanh = nn.Tanh()
        
    #hidden_state_sizes[0] is the size of the output of lstm 
    self.fc2 = nn.Linear(hidden_state_sizes[0] + meta_data_len, hidden_state_sizes[1])
        
    #hidden_state_sizes[1] is the size of the first and only hidden layer
    self.fc3 = nn.Linear(hidden_state_sizes[1], hidden_state_sizes[2])
    self.fc4 = nn.Linear(hidden_state_sizes[2],1)

        
  def forward(self, x, meta_data, hidden):
    batch_size = x.size(0)
    x = x.long()
    embeds = self.embedding(x)
    gru_out, hidden = self.Gate(embeds, hidden)
    gru_out = gru_out.contiguous().view(-1, self.hidden_dim)
    
    out = self.dropout(gru_out)
    out = self.fc1(out)

    out = out.view(batch_size, -1, self.hidden_dim)
    out = out[:,-1, :] 

    # combine hidden state and meta_data
    ################# ################# #################
    out = torch.cat((out, meta_data), dim = 1) #meta_data is of shape (batch_size, -1)
        
    out = self.fc2(out)
        
    # applying dropout before relu since relu already sets some neurons to 0
    out = self.dropout(out)

    out = self.sigmoid(out)

    out = self.fc3(out)

    out = self.relu(out)

    out = self.fc4(out)
        
    return out, hidden
    
  def init_hidden(self, batch_size):
    weight = next(self.parameters()).data
    n = 1
    if self.bidirectional == True: 
      n = 2
    if self.GRU_val == False:
      return (weight.new(self.n_layers * n, batch_size, self.hidden_dim).zero_().to('cuda'),
              weight.new(self.n_layers * n, batch_size, self.hidden_dim).zero_().to('cuda'))
    else:
      return  weight.new(self.n_layers * n, batch_size, self.hidden_dim).zero_().to('cuda')


def train_retweet_predictor(model, epochs = 100, print_every = 5, clip = 5, valid_loss_min = np.Inf, lr=0.005, batch_size = 400, device = 'cuda', GRU = False, weight_decay = 1e-5): 
  counter = 0
  print_every = 200
  model.train()
    
  criterion = nn.L1Loss()
    
  # weight decay is the l2 regularization penalty 
  optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)


  training_stats = []
  for i in range(epochs):
    total_train_loss = 0

    h = model.init_hidden(batch_size)
    for tweets, meta_data, labels in train_loader:
      counter += 1
      if GRU == False: 
        h = tuple([each.data for each in h])
      else:
        h = h.data
      tweets, meta_data, labels = tweets.to(device), meta_data.to(device), labels.to(device)
      model.zero_grad()
      output, h = model(tweets, meta_data, h)
      loss = criterion(output.squeeze(), labels.float())
      loss.backward()
      total_train_loss += loss.item()
      nn.utils.clip_grad_norm_(model.parameters(), clip)
      optimizer.step()

      if i % 10 == True:
        print("Epoch: {}/{}...".format(i+1, epochs),
            "Step: {}...".format(counter),
            "Loss: {:.6f}...".format(loss.item()))

In [75]:
vocab_size = 1410
output_size = 1
embedding_dim = 100
hidden_dim = 256
n_layers = 2

net = RetweetNet(vocab_size = vocab_size, hidden_state_sizes = [256,128, 64], meta_data_len = 4, output_size = output_size, embedding_dim = embedding_dim, hidden_dim  = hidden_dim, 
                 n_layers =n_layers, drop_prob = .1,  GRU = False, bidirectional = False)
net.cuda()

RetweetNet(
  (embedding): Embedding(1410, 100)
  (Gate): LSTM(100, 256, num_layers=2, batch_first=True, dropout=0.1)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc1): Linear(in_features=256, out_features=256, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
  (tanh): Tanh()
  (fc2): Linear(in_features=260, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=1, bias=True)
)

In [76]:
train_retweet_predictor(net, epochs = 150, batch_size = 25, device = 'cuda', lr = .001, GRU = False, weight_decay = 1e-04)

Epoch: 2/150... Step: 46... Loss: 10706.245117...
Epoch: 2/150... Step: 47... Loss: 8281.454102...
Epoch: 2/150... Step: 48... Loss: 10185.053711...
Epoch: 2/150... Step: 49... Loss: 7472.974609...
Epoch: 2/150... Step: 50... Loss: 8220.174805...
Epoch: 2/150... Step: 51... Loss: 1341.838257...
Epoch: 2/150... Step: 52... Loss: 5258.966309...
Epoch: 2/150... Step: 53... Loss: 9205.649414...
Epoch: 2/150... Step: 54... Loss: 6166.095703...
Epoch: 2/150... Step: 55... Loss: 8477.639648...
Epoch: 2/150... Step: 56... Loss: 3476.604004...
Epoch: 2/150... Step: 57... Loss: 2963.919678...
Epoch: 2/150... Step: 58... Loss: 6368.292480...
Epoch: 2/150... Step: 59... Loss: 1665.473999...
Epoch: 2/150... Step: 60... Loss: 16317.300781...
Epoch: 2/150... Step: 61... Loss: 7803.681641...
Epoch: 2/150... Step: 62... Loss: 4742.344238...
Epoch: 2/150... Step: 63... Loss: 3038.927002...
Epoch: 2/150... Step: 64... Loss: 5759.161133...
Epoch: 2/150... Step: 65... Loss: 5997.177246...
Epoch: 2/150... S

In [301]:
len(test_loader)

1125

In [304]:
test_data[0]

AttributeError: ignored

In [313]:
len(predictions_MA_test)

205

In [69]:
h = net.init_hidden(1)

In [77]:
train_data = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(X_train_meta))
train_loader = DataLoader(train_data, shuffle=False, batch_size=1)

predictions_MA_train = []
h = net.init_hidden(1)
for x,y in train_loader:
  output, h = net(x.to('cuda'), y.to('cuda'), h)
  predictions_MA_train.append(output.item())

In [328]:
len(predictions_exp_test, predictions_exp_train)

205

In [78]:
predictions_MA_test = []
h = net.init_hidden(1)
for step, (x,y) in enumerate(test_loader):
  output, h = net(x.to('cuda'), y.to('cuda'), h)
  print(output)
  predictions_MA_test.append(output.item())

"""train_data = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(X_train_meta))
train_loader = DataLoader(train_data, shuffle=False, batch_size=1)

predictions_MA_train = []
h = net.init_hidden(1)
for x,y in train_loader:
  output, h = net(x.to('cuda'), y.to('cuda'), h)
  predictions_MA_train.append(output.item())"""

tensor([[-482.2847]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[-1278.9703]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[-1396.4617]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[-191.3319]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[-186.8174]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[-162.9953]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[-1672.2944]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[-1475.2291]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[-130.0592]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[-137.0039]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[-111.9932]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[-149.7578]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[-1378.2390]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[-129.6238]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[-657.7313]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[-137.0029]]

"train_data = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(X_train_meta))\ntrain_loader = DataLoader(train_data, shuffle=False, batch_size=1)\n\npredictions_MA_train = []\nh = net.init_hidden(1)\nfor x,y in train_loader:\n  output, h = net(x.to('cuda'), y.to('cuda'), h)\n  predictions_MA_train.append(output.item())"

In [212]:
train_data = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(X_train_meta))
train_loader = DataLoader(train_data, shuffle=False, batch_size=1)

predictions_MA_train = []
h = net.init_hidden(1)
for x,y in train_loader:
  output, h = net(x.to('cuda'), y.to('cuda'), h)
  predictions_MA_train.append(output.item())

In [79]:
pd.Series(predictions_exp_test + predictions_exp_train).to_frame().to_csv('predicted_errors_exp.csv', index = False)

In [330]:
test = pd.read_csv('predicted_errors_exp.csv')

In [331]:
test.size

1330

In [234]:
len(predictions_MA_test)

1125

In [160]:
predictions[200]

tensor([[-0.0093]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [147]:
def predictions(model, batch_size = 205, device = 'cuda', GRU = False): 
  test_losses = []
  num_correct = 0
  model.cuda()
  model.freeze()
  predictions = []

  h = model.init_hidden(batch_size)
  
  model.eval()
  for tweets, meta_data in test_loader:
    if GRU == True: 
      h = h.data
    else: 
      h = tuple([each.data for each in h])
    tweets, meta_data = tweets.to(device), meta_data.to(device)
    output, h = model(tweets, meta_data, h)
    predictions.append(output)